## Image Captioning

In [3]:
!pip install transformers


In [4]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import cv2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
processor = BlipProcessor.from_pretrained("noamrot/FuseCap")
model1 = BlipForConditionalGeneration.from_pretrained("noamrot/FuseCap").to(device)

c:\Users\sapta\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [5]:
# img_url="https://cdn.dribbble.com/users/5054637/screenshots/14026155/media/7cc1339278e8a82be1cd66a7f9c5f6d2.jpeg"
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# raw_image=cv2.imread("WIN_20241113_16_35_15_Pro.jpg")
raw_image=cv2.imread("Images/3.jpg")
text = "describe the image"
inputs = processor(raw_image, text, return_tensors="pt").to(device)

out = model1.generate(**inputs, num_beams = 3)
corpus1= processor.decode(out[0], skip_special_tokens=True)
print(corpus1)

describe the image features a collection of blue bottles, including a coca - cola bottle, arranged in a row from left to right the bottles are labeled with the brand ' s name and logo,


## OCR

In [6]:
!pip install paddleocr

In [7]:
from paddleocr import PaddleOCR, draw_ocr # main OCR dependencies
from matplotlib import pyplot as plt # plot images
import cv2 #opencv
import os # folder directory navigation

In [8]:
ocr_model = PaddleOCR(lang='en')


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to C:\Users\sapta/.paddleocr/whl\cls\ch_ppocr_mobile_v2.0_cls_infer\ch_ppocr_mobile_v2.0_cls_infer.tar
[2024/11/13 20:05:02] ppocr INFO: Path C:\Users\sapta/.paddleocr/whl\cls\ch_ppocr_mobile_v2.0_cls_infer\ch_ppocr_mobile_v2.0_cls_infer.tar already exists. Skipping...
[2024/11/13 20:05:02] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\sapta/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sas

In [9]:
result=ocr_model.ocr("Images/3.jpg")

[2024/11/13 20:05:04] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/11/13 20:05:04] ppocr DEBUG: dt_boxes num : 13, elapsed : 0.691596508026123
[2024/11/13 20:05:05] ppocr DEBUG: rec_res num  : 13, elapsed : 0.7526006698608398


In [10]:
corpus = []
if result[0]!=None:
    for item in result[0]:
        text = item[1][0]  # Access the text part in each tuple
        corpus.append(text) # Append to corpus list

    # Join the list items into a single string for the corpus if needed
    corpus_text = ' '.join(corpus)
else:
    corpus_text=''

corpus_text

'THE ~MEDIA ANT Coca-Cola Find even more Anna nameswith Aart #ShareaCoke Rive Um Awr www.themediaant.com'

## Mistral model for labelling using API

In [11]:
!pip install mistralai

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.9.0.post0
    Uninstalling python-dateutil-2.9.0.post0:
      Successfully uninstalled python-dateutil-2.9.0.post0


In [14]:
coprus3="get your coke at flat 50 off today"

In [15]:
import os
from mistralai import Mistral

api_key = "iBGDfsGLNlGIr26A09CCttgSkaCjDwEM"
model = "mistral-large-latest"
client = Mistral(api_key=api_key)



chat_response = client.chat.complete(
    model=model,
    messages=[
        {
            "role": "user",
            "content": (
                "You are a product classifier chatbot. Given the following inputs, identify if it describes a product, "
                "and if so, provide the product details including name, price, size, weight, description, and subcategory.\n\n"
                f"1. OCR Text: {corpus1}\n"
                f"2. Image Caption: {corpus_text}\n"
                f"3. Instagram Caption: {coprus3}\n\n"
                "If it is a product, respond with the following format:\n"
                "- **Product Name**: \n"
                "- **Colour**: \n"
                "- **Price**: \n"
                "- **Size**: \n"
                "- **Weight**: \n"
                "- **Description**: \n"
                "- **Subcategory**: \n\n"
                "If it’s not a product, simply respond 'Not a product.'"
            ),
        },
    ]
)


print(chat_response.choices[0].message.content)


- **Product Name**: Coca-Cola Bottle
- **Colour**: Blue
- **Price**: 50% off (actual price not specified)
- **Size**: Not specified
- **Weight**: Not specified
- **Description**: A collection of blue Coca-Cola bottles, arranged in a row, labeled with the brand's name and logo.
- **Subcategory**: Beverages, Soft Drinks
